# AutoML with Vertex AI - Python Clients

This notebook uses Python Clients for Vertex AI to recreate this workflow:
- create a Dataset linked to BigQuery table
- Train an AutoML model for the Dataset
- Deploy the Model to and Endpoint
- Create Batch Predictions
- Create Online Predictions

**Prerequisites**
- `00 - Initial Setup`

**Resources**
- API: https://googleapis.dev/python/aiplatform/latest/aiplatform.html

**Overview**

<img src="architectures/statmike-mlops-A2.png">

---
## Setup

Import Libraries

In [1]:
from google.cloud import aiplatform

Parameters

In [2]:
# Locations
REGION = 'us-central1'
PROJECT_ID='statmike-mlops'
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

In [3]:
aiplatform.init(project=PROJECT_ID, location=REGION)

In [9]:
from datetime import datetime
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

---
## Code

In [5]:
dataset = aiplatform.TabularDataset.create(display_name='bq_digits_code', bq_source='bq://statmike-mlops.digits.digits_source')

INFO:google.cloud.aiplatform.datasets.dataset:Creating TabularDataset
INFO:google.cloud.aiplatform.datasets.dataset:Create TabularDataset backing LRO: projects/691911073727/locations/us-central1/datasets/3579561259294523392/operations/5385608338841010176
INFO:google.cloud.aiplatform.datasets.dataset:TabularDataset created. Resource name: projects/691911073727/locations/us-central1/datasets/3579561259294523392
INFO:google.cloud.aiplatform.datasets.dataset:To use this TabularDataset in another session:
INFO:google.cloud.aiplatform.datasets.dataset:ds = aiplatform.TabularDataset('projects/691911073727/locations/us-central1/datasets/3579561259294523392')


In [27]:
transformations = []

for i in range(64):
    transformations.append({'auto': {'column_name': 'p'+str(i)}})

In [28]:
tabular_classification_job = aiplatform.AutoMLTabularTrainingJob(display_name='bq_digits_code_'+TIMESTAMP, optimization_prediction_type = 'classification', column_transformations=transformations)

In [29]:
model = tabular_classification_job.run(
        dataset=dataset,
        target_column='target',
        training_fraction_split=0.8,
        validation_fraction_split=0.1,
        test_fraction_split=0.1,
        budget_milli_node_hours=1000,
        model_display_name='bq_digits_code_'+TIMESTAMP,
        disable_early_stopping=False,
        sync=True,
    )

INFO:google.cloud.aiplatform.training_jobs:View Training:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/3915092625673158656?project=691911073727
INFO:google.cloud.aiplatform.training_jobs:AutoMLTabularTrainingJob projects/691911073727/locations/us-central1/trainingPipelines/3915092625673158656 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aiplatform.training_jobs:AutoMLTabularTrainingJob projects/691911073727/locations/us-central1/trainingPipelines/3915092625673158656 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aiplatform.training_jobs:AutoMLTabularTrainingJob projects/691911073727/locations/us-central1/trainingPipelines/3915092625673158656 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aiplatform.training_jobs:AutoMLTabularTrainingJob projects/691911073727/locations/us-central1/trainingPipelines/3915092625673158656 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud

In [30]:
model

resource name: projects/691911073727/locations/us-central1/models/6423689977095258112